# Real Estate Price Prediction Moscow

![title](https://upload.wikimedia.org/wikipedia/commons/8/85/Saint_Basil%27s_Cathedral_and_the_Red_Square.jpg)

## 1. Описание датасета

##### File descriptions  
  train.csv - the training set  
  test.csv - the test set  
  sampleSubmission.csv - a sample submission file in the correct format  
##### Data fields
Id - идентификационный номер квартиры  
DistrictId - идентификационный номер района  
Rooms - количество комнат  
Square - площадь  
LifeSquare - жилая площадь  
KitchenSquare - площадь кухни  
Floor - этаж  
HouseFloor - количество этажей в доме  
HouseYear - год постройки дома  
Ecology_1, Ecology_2, Ecology_3 - экологические показатели местности  
Social_1, Social_2, Social_3 - социальные показатели местности  
Healthcare_1, Helthcare_2 - показатели местности, связанные с охраной здоровья  
Shops_1, Shops_2 - показатели, связанные с наличием магазинов, торговых центров  
Price - цена квартиры  

## 2. Импорт библиотек и настройка основных параметров

In [11]:
import pandas as pd
import matplotlib
import matplotlib.image as img
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [12]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
matplotlib.rcParams.update({'font.size': 14})

In [13]:
pd.options.display.max_columns = 100

In [14]:
DATASET_TRAIN_PATH = 'C:/Users/Gagik Vardanyan/Desktop/Jupyter/DataScience Course work/real-estate-price-prediction-moscow/train.csv'
TEST_DATASET_PATH = 'C:/Users/Gagik Vardanyan/Desktop/Jupyter/DataScience Course work/real-estate-price-prediction-moscow/test.csv'

In [15]:
df = pd.read_csv(DATASET_TRAIN_PATH, sep=",")
df

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Gagik Vardanyan/Desktop/Jupyter/DataScience Course work/real-estate-price-prediction-moscow/train.csv'

Загрузим тестовый датасет

Тестовый датасет содержит 20 различных признаков, включая ID квартиры

Применим функцию для уменьшения размерности типа данных и объема используемой памяти

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
df = reduce_mem_usage(df)
df.dtypes

Объем используемой памяти уменьшился на 68.1%

## 3. Exploratory Data Analysis

Посмотрим на распределение целевой переменной

In [ ]:
plt.figure(figsize = (16, 8))

df['Price'].hist(bins=100)
plt.ylabel('Count')
plt.xlabel('Price')

plt.title('Target distribution')
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [14, 3.50]
plt.rcParams["figure.autolayout"] = True
plt.title('Target distribution')
sns.boxplot(data=df, x="Price", orient="h", notch=True, showcaps=False,
    flierprops={"marker": "x"},
    boxprops={"facecolor": (.4, .6, .8, .5)},
    medianprops={"color": "coral"},)

In [ ]:
df.Price.describe()

Мы можем сделать заключение, что у исходного датасета средняя (mean) цена за квартиру 214138$,   
`25% квартиль - 153872$`  
`50% квартиль - 192269$`  
`75% квартиль - 249135$`  

Проверим, есть ли в датасете пропуски данных

In [ ]:
df.info()

Признаки *LifeSquare*, *Healthcare_1* содержат пропуски. Необходимо будет поработать над ними

### Выбросим признак "ID" чтобы алгоритм не учился на номере объекта

In [ ]:
df.drop(columns = ['Id'], axis = 1, inplace = True)
df.head()

### Начнем обработку датасета с пропусков

Еще раз проверим в каких признаках и сколько имеется пропусков

In [ ]:
df.isna().sum()

In [ ]:
df['LifeSquare_nan'] = 0
df.loc[df['LifeSquare'].isna(), 'LifeSquare_nan'] = 1
df.head()

Попробуем обработать пропуски LifeSquare не по средней, а через средний коэффициент соотношения жилой площади к нежилой

In [ ]:
SqL = df.loc[df['LifeSquare'].notnull()]
SqL['SqLRatio'] = SqL['Square'] / SqL['LifeSquare']
SqL['SqLRatio'].mean

In [ ]:
df['LifeSquare'] = df['LifeSquare'].fillna(df['Square']/1.7192)

##### Проверим, что пропуски отсутствуют

In [ ]:
df.isna().sum()

##### C Healthcare_1 сложнее, так как непонятно, что из себя представляет эта величина. Попробуем через медиану

In [ ]:
df['Healthcare_1'].median()

In [ ]:
df['Healthcare_1_nan'] = 0
df.loc[df['Healthcare_1'].isna(), 'Healthcare_1_nan'] = 1
df.head()

In [ ]:
df['Healthcare_1'] = df['Healthcare_1'].fillna(900)
df.head(5)

### Обработаем выбросы

#### Посмотрим на данные

In [ ]:
df.describe()

In [ ]:
df_num_features = df.select_dtypes(include=['float64', 'float32', 'float16', 'int64', 'int32', 'int16', 'int8'])
df_num_features.head()

In [ ]:
df_num_features.hist(figsize=(16, 16), bins=20, grid=False);

#### Посмотрим на параметр Rooms

In [ ]:
df['Rooms'].value_counts()

In [ ]:
df.loc[df['Rooms'].isin([0, 10, 19]), 'Rooms'] = df['Rooms'].median()

#### Посмотрим на параметр Square, LifeSquare

In [ ]:
Square_min_treshold, Square_max_treshhold = df['Square'].quantile([0.01, 0.99])
Square_min_treshold, Square_max_treshhold

In [ ]:
LifeSquare_min_treshold, LifeSquare_max_treshhold = df['LifeSquare'].quantile([0.01, 0.99])
LifeSquare_min_treshold, LifeSquare_max_treshhold

In [ ]:
df = df[(df['Square'] < df['Square'].quantile(.99)) & (df['Square'] > df['Square'].quantile(.01))]
df = df[(df['LifeSquare'] < df['LifeSquare'].quantile(.99)) & (df['LifeSquare'] > df['LifeSquare'].quantile(.01))]

#### Параметр KitchenSquare не будем отбрасывать, заменим неправдоподобно низкие значения на выбранное

In [ ]:
df['KitchenSquare'].sort_values().unique()

In [ ]:
df.loc[df['KitchenSquare'] < 3, 'KitchenSquare'] = 3

In [ ]:
df.loc[df['KitchenSquare'] > 84, 'KitchenSquare'] = 84

#### Параметр Floor, House Floor обработаем

In [ ]:
df['Floor'].sort_values().unique()

In [ ]:
df['HouseFloor'].sort_values().unique()

In [ ]:
df.loc[df['HouseFloor'] == 0, 'HouseFloor'] = df['HouseFloor'].median()

In [ ]:
false_floor = df.loc[df['Floor'] > df['HouseFloor']].index

In [ ]:
df.loc[false_floor, 'Floor'] = df.loc[false_floor, 'HouseFloor'].apply(lambda x: x)

#### Параметр HouseYear

In [ ]:
df['HouseYear'].sort_values(ascending = False).unique()

In [ ]:
df.loc[df['HouseYear'] > 2022, 'HouseYear'] = 2020

### Получим новые параметры

#### Переведем тип object в Dummies

In [ ]:
pd.get_dummies(df['Ecology_2'])
df = pd.concat([df, pd.get_dummies(df['Ecology_2'])], axis=1)
df = df.rename(columns = {'A' : 'Ecology_2_A', 'B' : 'Ecology_2_B'})
pd.get_dummies(df['Ecology_3'])
df = pd.concat([df, pd.get_dummies(df['Ecology_3'])], axis=1)
df = df.rename(columns = {'A' : 'Ecology_3_A', 'B' : 'Ecology_3_B'})
pd.get_dummies(df['Shops_2'])
df = pd.concat([df, pd.get_dummies(df['Shops_2'])], axis=1)
df = df.rename(columns = {'A' : 'Shops_2_A', 'B' : 'Shops_2_B'})
df.head()

#### Внесем новые показатели: 

In [ ]:
def house_age_cat(X):

    X['house_age_cat'] = 0

    X.loc[X['HouseYear'] <= 1980, 'house_age_cat'] = 1  
    X.loc[(X['HouseYear'] > 1980) & (X['HouseYear'] <= 2000), 'house_age_cat'] = 2
    X.loc[(X['HouseYear'] > 2000) & (X['HouseYear'] <= 2010), 'house_age_cat'] = 3
    X.loc[X['HouseYear'] > 2010, 'house_age_cat'] = 4

    return X

In [ ]:
df = house_age_cat(df)
df.head()

In [ ]:
df['Floor_ratio'] = df['Floor'] / df['HouseFloor']
df.head()

In [ ]:
df["SquareMeterPrice"] = df["Price"] / df["Square"]
SqMetPrice_dist = df.groupby('DistrictId', as_index=False).agg({'SquareMeterPrice': 'mean'})\
            .rename(columns={'SquareMeterPrice': 'AvSqMetPriceDist'})
df = df.merge(SqMetPrice_dist, on=['DistrictId'], how='left')

In [ ]:
Healthcare_1_dist = df.groupby('DistrictId', as_index=False)\
            .agg({'Healthcare_1': 'mean'})\
            .rename(columns={'Healthcare_1': 'AvHealthcareDist_1'})
df = df.merge(Healthcare_1_dist, on=['DistrictId'], how='left')

In [ ]:
median_price_dist = df.groupby(['DistrictId', 'Rooms'], as_index=False).agg({'Price':'median'})\
                            .rename(columns={'Price':'MedianPriceDistrict'})

df = df.merge(median_price_dist, on=['DistrictId', 'Rooms'], how='left')

In [ ]:
df

In [ ]:
df.isna().sum()

##### Еще раз посмотрим на наши признаки после обработки данных

In [ ]:
df_num_features = df[df.columns.drop(['Ecology_1', 'Ecology_2', 'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Shops_2', 'LifeSquare_nan', 'Healthcare_1_nan', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A', 'Shops_2_B'])]
df_num_features.head()

In [ ]:
df_num_features.hist(figsize=(16, 16), bins=20, grid=False);

#### Посмотрим на корреляционную матрицу, чтобы оценить можно ли не брать некоторые признаки

In [ ]:
plt.figure(figsize=(16, 10))

sns.set(font_scale=1.4)

corr_matrix = df.corr()
corr_matrix = np.round(corr_matrix, 2)
corr_matrix[np.abs(corr_matrix) < 0.3] = 0

sns.heatmap(corr_matrix, annot=True, linewidths=.5, cmap='coolwarm')

plt.title('Correlation matrix')
plt.show()

#### Отберем признаки

In [ ]:
train_df = df
train_df.head()

In [ ]:
feature_name = ['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear',
                'Ecology_1', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Social_1',
                'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Shops_2_A', 'Shops_2_B', 'house_age_cat', 
                'Floor_ratio', 'MedianPriceDistrict', 'AvSqMetPriceDist', 'AvHealthcareDist_1']
target_name = ['Price']

In [ ]:
X = train_df[feature_name]
y = train_df[target_name]

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
train_df = pd.read_csv(DATASET_TRAIN_PATH)
test_df = pd.read_csv(TEST_DATASET_PATH)

In [ ]:
train_df.shape

#### Загрузка дополнительных модулей

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2, mean_squared_error as mse
from sklearn.model_selection import KFold, GridSearchCV

#### Разбивка на Train & Test

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_valid.shape

##### Функция для оценки метрик модели

In [ ]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)) + "\n" +
          "RMSE:\t" + str(round(np.sqrt(mse(train_true_values, train_pred_values)), 3)) + "\n" +
          "MSE:\t" + str(round(mse(train_true_values, train_pred_values), 3)))
    print("********************************")
    print("R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)) + "\n" +
          "RMSE:\t" + str(round(np.sqrt(mse(test_true_values, test_pred_values)), 3)) + "\n" +
          "MSE:\t" + str(round(mse(test_true_values, test_pred_values), 3)))

##### Предварительно использовал GridSearchCV, но не стал включать ее в финальную версию ноутбука

RFR = RandomForestRegressor()

RFR.get_params().keys()

%%time
parameters = {'n_estimators' : [50,100,150,200,300],
              'max_depth'    : [3,4,5,6,7,8,9,10],
              'max_features' : [5, 7, 9, 11, 15, 20]
                 }

grid_RFR = GridSearchCV(estimator=RFR, param_grid = parameters, cv = 2, n_jobs=-1, scoring='r2')
grid_RFR.fit(X_train, y_train)

print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid_RFR.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_RFR.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_RFR.best_params_)

In [ ]:
rf_model_2 = RandomForestRegressor(max_depth = 10 , max_features = 9, n_estimators = 300, random_state=42)
rf_model_2.fit(X_train, y_train)

In [ ]:
y_train_preds = rf_model_2.predict(X_train)
y_test_preds = rf_model_2.predict(X_valid)

evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds)

##### Выполним кросс-валидацию

In [ ]:
cv_score = cross_val_score(rf_model_2, X_train, y_train, scoring='r2', cv=KFold(n_splits=3, shuffle=True, random_state=21))
cv_score

In [ ]:
cv_score.mean()

##### Определим Важность признаков

In [ ]:
feature_importances = pd.DataFrame(zip(X_train.columns, rf_model_2.feature_importances_), 
                                   columns=['feature_name', 'importance'])

feature_importances.sort_values(by='importance', ascending=False)

### Прогнозирование на тестовом датасете

#### Подготовим данные

In [ ]:
test_df.shape

In [ ]:
test_df.head()

In [ ]:
test_df = reduce_mem_usage(test_df)
test_df.dtypes

In [ ]:
test_df.drop(columns = ['Id'], axis = 1, inplace = True)

In [ ]:
test_df['LifeSquare_nan'] = 0
test_df.loc[df['LifeSquare'].isna(), 'LifeSquare_nan'] = 1
test_df

In [ ]:
SqL = test_df.loc[test_df['LifeSquare'].notnull()]
SqL['SqLRatio'] = SqL['Square'] / SqL['LifeSquare']
SqL['SqLRatio'].mean

In [ ]:
test_df['LifeSquare'] = test_df['LifeSquare'].fillna(test_df['Square']/1.929783)

In [ ]:
test_df['Healthcare_1'].median()

In [ ]:
test_df['Healthcare_1_nan'] = 0
test_df.loc[test_df['Healthcare_1'].isna(), 'Healthcare_1_nan'] = 1
test_df.head()

In [ ]:
test_df['Healthcare_1'] = test_df['Healthcare_1'].fillna(900)

In [ ]:
test_df['Rooms'].value_counts()

In [ ]:
test_df.loc[test_df['Rooms'].isin([0, 17]), 'Rooms'] = df['Rooms'].median()

In [ ]:
Square_min_treshold, Square_max_treshhold = test_df['Square'].quantile([0.01, 0.99])
Square_min_treshold, Square_max_treshhold

In [ ]:
LifeSquare_min_treshold, LifeSquare_max_treshhold = test_df['LifeSquare'].quantile([0.01, 0.99])
LifeSquare_min_treshold, LifeSquare_max_treshhold

In [ ]:
test_df['KitchenSquare'].sort_values().unique()

In [ ]:
test_df.loc[test_df['KitchenSquare'] < 3, 'KitchenSquare'] = 3
test_df.loc[test_df['KitchenSquare'] > 97, 'KitchenSquare'] = 97

In [ ]:
test_df['Floor'].sort_values().unique()

In [ ]:
test_df['HouseFloor'].sort_values().unique()

In [ ]:
test_df.loc[test_df['HouseFloor'] == 0, 'HouseFloor'] = test_df['HouseFloor'].median()
false_floor = test_df.loc[test_df['Floor'] > test_df['HouseFloor']].index
test_df.loc[false_floor, 'Floor'] = test_df.loc[false_floor, 'HouseFloor'].apply(lambda x: x)

In [ ]:
test_df['HouseYear'].sort_values(ascending = False).unique()

In [ ]:
pd.get_dummies(test_df['Ecology_2'])
test_df = pd.concat([test_df, pd.get_dummies(test_df['Ecology_2'])], axis=1)
test_df = test_df.rename(columns = {'A' : 'Ecology_2_A', 'B' : 'Ecology_2_B'})
pd.get_dummies(test_df['Ecology_3'])
test_df = pd.concat([test_df, pd.get_dummies(test_df['Ecology_3'])], axis=1)
test_df = test_df.rename(columns = {'A' : 'Ecology_3_A', 'B' : 'Ecology_3_B'})
pd.get_dummies(test_df['Shops_2'])
test_df = pd.concat([test_df, pd.get_dummies(test_df['Shops_2'])], axis=1)
test_df = test_df.rename(columns = {'A' : 'Shops_2_A', 'B' : 'Shops_2_B'})
test_df.head()

In [ ]:
test_df = house_age_cat(test_df)

In [ ]:
test_df['Floor_ratio'] = test_df['Floor'] / test_df['HouseFloor']

In [ ]:
test_df

In [ ]:
median_price_dist = df.groupby(['DistrictId', 'Rooms'], as_index=False).agg({'Price':'median'})\
                            .rename(columns={'Price':'MedianPriceDistrict'})

In [ ]:
median_price_dist

In [ ]:
test_df = test_df.merge(median_price_dist, on=['DistrictId', 'Rooms'], how='left')

In [ ]:
test_df = test_df.merge(SqMetPrice_dist, on=['DistrictId'], how='left')

In [ ]:
test_df = test_df.merge(Healthcare_1_dist, on=['DistrictId'], how='left')

In [ ]:
test_df

In [ ]:
test_df.isnull().sum()

In [ ]:
median_1 = test_df['MedianPriceDistrict'].median()

In [ ]:
test_df['MedianPriceDistrict'] = test_df['MedianPriceDistrict'].fillna(median_1)

In [ ]:
median_2 = test_df['AvSqMetPriceDist'].median()
test_df['AvSqMetPriceDist'] = test_df['AvSqMetPriceDist'].fillna(median_2)

In [ ]:
median_3 = test_df['AvHealthcareDist_1'].median()
test_df['AvHealthcareDist_1'] = test_df['AvHealthcareDist_1'].fillna(median_3)

In [ ]:
test_df.isnull().sum()

#### Отберем необходимые признаки и сделаем предсказание на обученной модели

In [ ]:
feature_name = ['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear',
                'Ecology_1', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Social_1',
                'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Shops_2_A', 'Shops_2_B', 'house_age_cat', 
                'Floor_ratio', 'MedianPriceDistrict', 'AvSqMetPriceDist', 'AvHealthcareDist_1']
target_name = ['Price']

In [ ]:
X = test_df[feature_name]

In [ ]:
test_predict = rf_model_2.predict(X)

In [ ]:
test_predict

In [ ]:
submit = pd.read_csv('C:/Users/Gagik Vardanyan/Desktop/Jupyter/DataScience Course work/real-estate-price-prediction-moscow/sample_submission.csv')
submit.head()

In [ ]:
submit['Price'] = test_predict
submit.head()

In [ ]:
submit.to_csv('VG_prediction_01.csv', index=False)